In [23]:
!pip install -qU "langchain[google-vertexai]"
!pip install langchain-core langgraph>0.2.27



In [2]:
from google.colab import auth

auth.authenticate_user()

In [3]:
PROJECT_ID = "automatation-trading"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
STAGING_BUCKET = "gs://llm-mariano"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

In [36]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash-001", model_provider="google_vertexai")

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 9, 'candidates_token_count': 3, 'total_token_count': 12, 'prompt_tokens_details': [{'modality': 1, 'token_count': 9}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 3}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.012654953946669897}, id='run-800cbd28-af88-4807-bfdc-343a2a6994b2-0', usage_metadata={'input_tokens': 9, 'output_tokens': 3, 'total_tokens': 12})

In [6]:
for token in model.stream(messages):
    print(token.content, end="|")

Ciao|!
|

In [7]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [8]:
prompt = prompt_template.invoke({"language": "Italian", "text": "hi!"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

In [9]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

In [17]:
response = model.invoke(prompt)
print(response.content)

Ciao!



In [37]:
model.invoke([HumanMessage("What is my name?")])

AIMessage(content='I am an AI and do not know your name.\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 5, 'candidates_token_count': 12, 'total_token_count': 17, 'prompt_tokens_details': [{'modality': 1, 'token_count': 5}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 12}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.3538728952407837}, id='run-bc70d341-3602-400a-b14b-7cad4b792e69-0', usage_metadata={'input_tokens': 5, 'output_tokens': 12, 'total_tokens': 17})

In [20]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob. You just told me! 😉\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 22, 'candidates_token_count': 12, 'total_token_count': 34, 'prompt_tokens_details': [{'modality': 1, 'token_count': 22}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 12}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.2570093274116516}, id='run-49a25575-7c80-483b-9d46-4f3e4eabb0bc-0', usage_metadata={'input_tokens': 22, 'output_tokens': 12, 'total_tokens': 34})

# Checkpoint

In [72]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.graph.message import add_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage

from typing import TypedDict, Annotated, Sequence

class State(TypedDict):
  messages: Annotated[Sequence[BaseMessage], add_messages]
  language: str


workflow = StateGraph(state_schema=State)


prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

workflow.add_edge(START,"model")
workflow.add_node("model",call_model)

memory_saver = MemorySaver()
app = workflow.compile(checkpointer=memory_saver)


In [73]:
config = {"configurable": {"thread_id": "we"}}

In [76]:
query = "Hola Soy  Bob"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages, "language":"Spanish"}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Ah, con que "Bob" es tu nombre! ¡Un placer conocerte, marinero Bob! ¿Qué te trae a mi bordo? ¿Tienes alguna pregunta para este viejo lobo de mar?


In [77]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Jajaja! ¡Qué despistado estás, Bob! ¡Acabas de decirme que te llamas Bob! ¡Soy un viejo lobo de mar, no un loro! ¿Necesitas un trago de grog para refrescar la memoria?


# Reasoning engine
The Reasoning Engine API provides the managed runtime for your customized agentic workflows in generative AI applications. You can create an application using orchestration frameworks such as LangChain, and deploy it with Reasoning Engine. This service has all the security, privacy, observability, and scalability benefits of Vertex AI integration.

In [11]:
from vertexai.preview import reasoning_engines


In [12]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

In [13]:
agent = reasoning_engines.LangchainAgent(
    model= "gemini-1.5-pro",
    tools=[get_exchange_rate],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [14]:
response = agent.query(input="What's the exchange rate from US dollars to Euros currency today?")


In [15]:
response['output']

'The exchange rate from US dollars to Euros is 0.96367. That means for every 1 US dollar, you would get 0.96367 Euros.'

In [16]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[langchain,reasoningengine]",
        "cloudpickle==3.0.0",
        "pydantic>=2.10",
        "requests",
    ],
)

INFO:vertexai.reasoning_engines._reasoning_engines:Using bucket llm-mariano
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://llm-mariano/reasoning_engine/reasoning_engine.pkl
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://llm-mariano/reasoning_engine/requirements.txt
INFO:vertexai.reasoning_engines._reasoning_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://llm-mariano/reasoning_engine/dependencies.tar.gz
INFO:vertexai.reasoning_engines._reasoning_engines:Creating ReasoningEngine
INFO:vertexai.reasoning_engines._reasoning_engines:Create ReasoningEngine backing LRO: projects/571915015654/locations/us-central1/reasoningEngines/934057118028267520/operations/3401376500841185280


InternalServerError: 500 
 Please navigate to the Cloud Console Log Explorer page (https://console.cloud.google.com/logs/query;query=resource.type="aiplatform.googleapis.com%2FReasoningEngine"%0Aresource.labels.reasoning_engine_id=~"934057118028267520"?project=571915015654) to view the specific errors. Additionally, please check our troubleshooting guide (https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/troubleshooting/deploy) for more information, or visit our Cloud community forum (https://www.googlecloudcommunity.com/gc/AI-ML/bd-p/cloud-ai-ml) or GitHub repository (https://github.com/googleapis/python-aiplatform/issues) to find answers and ask for help. 13: 
 Please navigate to the Cloud Console Log Explorer page (https://console.cloud.google.com/logs/query;query=resource.type="aiplatform.googleapis.com%2FReasoningEngine"%0Aresource.labels.reasoning_engine_id=~"934057118028267520"?project=571915015654) to view the specific errors. Additionally, please check our

In [ ]:

remote_agent.query(
    input="What's the exchange rate from US dollars to Swedish currency today?"
)